In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_29204\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_29204\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,Sugarcane,28-28
1,4,Millets,28-28
2,4,Millets,17-17-17
3,4,Barley,10-26-26
4,3,Paddy,DAP


In [11]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [12]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,8,28-28
1,4,4,28-28
2,4,4,17-17-17
3,4,0,10-26-26
4,3,6,DAP


In [13]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [14]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,8,4
1,4,4,4
2,4,4,2
3,4,0,0
4,3,6,5


In [15]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [16]:
df_train.tail()

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil Type,Crop Type,Fertilizer Name
749995,749995,25,69,30,8,16,6,1,3,4
749996,749996,37,64,58,38,8,20,2,8,2
749997,749997,35,68,59,6,11,29,4,2,0
749998,749998,31,68,29,9,11,12,3,1,3
749999,749999,33,55,29,4,10,9,2,1,6


In [17]:
matrix=df_train.corr()

In [18]:
matrix

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil Type,Crop Type,Fertilizer Name
id,1.000000,-0.001069,-0.000310,0.001752,0.001505,-0.000375,-0.001234,0.002831,-0.000953,-0.000179
Temparature,-0.001069,1.000000,0.001758,-0.001633,-0.006073,-0.009913,0.000298,-0.007621,-0.006753,0.001727
Humidity,-0.000310,0.001758,1.000000,0.003424,0.006502,0.005229,0.002413,0.003921,0.002414,0.010626
Moisture,0.001752,-0.001633,0.003424,1.000000,0.003016,-0.001931,-0.006735,-0.003941,-0.003400,0.011493
Nitrogen,0.001505,-0.006073,0.006502,0.003016,1.000000,0.000543,-0.005464,-0.000650,0.001611,-0.004725
Potassium,-0.000375,-0.009913,0.005229,-0.001931,0.000543,1.000000,0.000113,0.005314,-0.003748,-0.010872
Phosphorous,-0.001234,0.000298,0.002413,-0.006735,-0.005464,0.000113,1.000000,-0.000275,0.000694,-0.006127
Soil Type,0.002831,-0.007621,0.003921,-0.003941,-0.000650,0.005314,-0.000275,1.000000,-0.006213,0.016576
Crop Type,-0.000953,-0.006753,0.002414,-0.003400,0.001611,-0.003748,0.000694,-0.006213,1.000000,-0.013041
Fertilizer Name,-0.000179,0.001727,0.010626,0.011493,-0.004725,-0.010872,-0.006127,0.016576,-0.013041,1.000000


In [19]:
import tensorflow as tf

In [20]:
from tensorflow import keras

In [21]:
X = df_train[["Nitrogen","Potassium","Phosphorous","Temparature","Humidity","Moisture","Soil Type"]]

In [22]:
y = df_train[["Fertilizer Name"]]

In [23]:
from tensorflow.keras.optimizers import RMSprop

In [24]:
from sklearn.preprocessing import MinMaxScaler


In [25]:
import pandas as pd
import numpy as np

def create_classwise_batches(df_train, Soil_Type_col, batch_size):
    batches = []
    grouped = df.groupby('Soil Type')

    for class_label, group in grouped:
        group = group.sample(frac=1).reset_index(drop=True)  # Shuffle
        for i in range(0, len(group), batch_size):
            batch = group.iloc[i:i + batch_size]
            batches.append(batch)

    return batches

# Example usage
df = df_train
batches = create_classwise_batches(df, Soil_Type_col='Soil Type', batch_size=32)

# Access a batch
First_batch = batches[1]

In [26]:
Soil_Type_col = 'Soil Type'  # Make sure this matches exactly
grouped = df_train.groupby(Soil_Type_col)

In [27]:
print(f'Total batches created: {len(batches)}')
print(batches[20].head(10))  # Preview the first batch

Total batches created: 23440
         id  Temparature  Humidity  Moisture  Nitrogen  Potassium  \
640   38387           29        68        65        12         11   
641  476400           33        71        60        25          0   
642  181932           36        58        38         4         18   
643  491716           35        56        37        39         17   
644  560007           38        61        36        29         15   
645  639162           38        52        36        31         15   
646  417887           27        50        38        13         14   
647  148280           35        72        36         9          9   
648  129814           32        53        34         5          6   
649  372450           31        56        61         6         15   

     Phosphorous  Soil Type  Crop Type  Fertilizer Name  
640           23          0          2                1  
641           20          0          8                0  
642           22          0          

In [28]:
X_batches = [batch.drop(columns=['Soil Type']) for batch in batches]
y_batches = [batch['Soil Type'] for batch in batches]

In [29]:
import numpy as np

X_batches_np = [x.values for x in X_batches]
y_batches_np = [y.values for y in y_batches]

In [30]:
import numpy as np
import tensorflow as tf

# Flatten the list of arrays into one big array
X_all = np.concatenate(X_batches_np, axis=0)
y_all = np.concatenate(y_batches_np, axis=0)

# Now create the dataset
dataset = tf.data.Dataset.from_tensor_slices((X_all, y_all))
dataset = dataset.shuffle(buffer_size=1024).batch(32, drop_remainder=True)

In [31]:
print(np.unique(y_all, return_counts=True))  # See if one class dominates

(array([0, 1, 2, 3, 4]), array([150956, 148382, 145850, 148102, 156710], dtype=int64))


In [32]:
print(type(X_batches_np))  # Should be <class 'list'>
print(type(X_batches_np[0]))  # Should be <class 'numpy.ndarray'>
print(X_batches_np[0].shape)  # Should be (batch_size, num_features)

<class 'list'>
<class 'numpy.ndarray'>
(32, 9)


In [33]:
import numpy as np

X_all = np.concatenate(X_batches_np, axis=0)
y_all = np.concatenate(y_batches_np, axis=0)

print(X_all.shape)  # Should be (total_samples, num_features)
print(y_all.shape)  # Should be (total_samples,)

(750000, 9)
(750000,)


In [34]:
X_all = np.array(X_all, dtype=np.float32)
y_all = np.array(y_all, dtype=np.int32)

In [35]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(buffer_size=1024).batch(32, drop_remainder=True)

In [36]:
X_np = X.to_numpy()  # Convert DataFrame to NumPy array
X_np.setflags(write=1)  # Ensure it's writable
X_np = X_np.astype(np.float32)  # Convert to desired dtype

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import SGD

model = Sequential([
    Dense(64, activation='relu', input_shape=(7,)),
    Dropout(0.3),
    Dense(32, activation='relu',kernel_regularizer=l2(0.05)),
    Dropout(0.3),
    Dense(10, activation='softmax')  # or 'sigmoid' for binary classification
])

optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.fit(dataset, epochs=10)

Epoch 1/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - accuracy: 0.1508 - loss: 1.9933
Epoch 2/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - accuracy: 0.1515 - loss: 1.9449
Epoch 3/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 51s 2ms/step - accuracy: 0.1514 - loss: 1.9442
Epoch 4/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - accuracy: 0.1521 - loss: 1.9439
Epoch 5/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - accuracy: 0.1526 - loss: 1.9436
Epoch 6/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step - accuracy: 0.1522 - loss: 1.9435
Epoch 7/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step - accuracy: 0.1531 - loss: 1.9433
Epoch 8/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - accuracy: 0.1527 - loss: 1.9432
Epoch 9/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 51s 2ms/step - accuracy: 0.1524 - loss: 1.9432
Epoch 10/10
23437/23437 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - accuracy: 0.1529 - loss: 1.9431


In [39]:
results = model.evaluate(dataset)
print(results)

23437/23437 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - accuracy: 0.1497 - loss: 1.9435
[1.9434595108032227, 0.14993919432163239]


In [52]:
# Sanity check on label spread
print(np.unique(y, return_counts=True))

# Try predicting from untrained model
sample_preds = model.predict(X[:26])
print("Predictions:", sample_preds.argmax(axis=1))

(array([0, 1, 2, 3, 4, 5, 6]), array([113887, 114436, 112453, 110889, 111158,  94860,  92317],
      dtype=int64))
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Predictions: [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
